In [ ]:
import cv2
import numpy as np
import torch
from torchvision.transforms import Compose, Normalize, ToTensor

%run GradCam.ipynb
%run GuidedBackProp.ipynb

def preprocess_image_GradCAM(img: np.ndarray, mean=None, std=None) -> torch.Tensor:
    if std is None:
        std = [0.5, 0.5, 0.5]
    if mean is None:
        mean = [0.5, 0.5, 0.5]

    preprocessing = Compose([
        ToTensor(),
        Normalize(mean=mean, std=std)
    ])

    return preprocessing(img.copy()).unsqueeze(0)

def deprocess_image_GradCAM(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img * 255)

def show_cam_on_image_GradCAM(img: np.ndarray, mask: np.ndarray) -> np.ndarray:
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

def gradcam_guidedbackprop_visualize_single_image(image, model, target_layer, device=None):
    '''
    @image: image in format of CHW or 1CHW, residing on CPU
    @model: network model
    @target_layer: the target layer to compute gradients
    @device: if to use GPU, the data will be transfered to device
    '''
    if device is not None:
        use_cuda = True
    else:
        use_cuda = False
        
    image_t = image.copy()
    image_t = np.float32(image_t) / 255
    image_t = preprocess_image_GradCAM(image_t, mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
    image = np.float32(image)/255

    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=use_cuda)
    target_category = None
    grayscale_cam = cam(input_tensor=image_t, target_category=target_category)
    
    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=use_cuda)
    gb = gb_model(image_t, target_category=target_category)
    image = image.squeeze()
    cam_image = show_cam_on_image_GradCAM(image, grayscale_cam)
    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image_GradCAM(cam_mask * gb)
    gb = deprocess_image_GradCAM(gb)

    viewer.imshow_(image, gray=False)
    viewer.imshow_(cam_image, gray=False)
    viewer.imshow_(cam_gb, gray=False)
    #viewer.imshow_(gb, gray=False)
    
def gradcam_guidedbackprop_visualize(image, model, target_layer, device=None):
    '''
    @image: image in format of CHW or 1CHW, residing on CPU
    @model: network model
    @target_layer: the target layer to compute gradients
    @device: if to use GPU, the data will be transfered to device
    '''
    if device is not None:
        use_cuda = True
    else:
        use_cuda = False

    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=use_cuda)

    if len(image.shape) == 3:
        image = image.unsqueeze(0)
    if device is not None:
        image = image.to(device)

    target_category = None
    grayscale_cam = cam(input_tensor=image, target_category=target_category)
    
    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=use_cuda)
    gb = gb_model(image, target_category=target_category)
    
    image = image.cpu().numpy().squeeze()
    image = np.moveaxis(image, 0, -1)
    cam_image = show_cam_on_image_GradCAM(image, grayscale_cam)
    
    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image_GradCAM(cam_mask * gb)
    gb = deprocess_image_GradCAM(gb)

    viewer.imshow_(image, gray=False)
    viewer.imshow_(cam_image, gray=False)
    viewer.imshow_(cam_gb, gray=False)
    #viewer.imshow_(gb, gray=False)